In [1]:
!pip install mediapipe --user

The system cannot find the path specified.


In [2]:
import cv2
import mediapipe as mp
import time

In [3]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import math
import numpy as np

In [ ]:
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=2,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

pTime = 0
cTime = 0

land_marks = []
max_dist = 0
min_stay = 0
while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    if cv2.waitKey(1) == ord('q'):
        cap.release()
        break
 
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for idd, lm in enumerate(handLms.landmark):
                
                h, w, c = img.shape
                cx, cy = int(lm.x *w), int(lm.y*h)
                #if id ==0:
                if idd == 4 or idd == 8:
                    land_marks.append((idd, [cx,cy]))
                    
                cv2.circle(img, (cx,cy), 3, (255,0,255), cv2.FILLED)

            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    try:
        x1, y1 = land_marks[-2][1]
        x2, y2 = land_marks[-1][1]
        dist = ((x2-x1)**2 + (y2-y1)**2)**0.5
    except:
        pass
    
    




    try:
        vol = min(1, round(dist/200, 2))
        cv2.putText(img, f"volume : {vol*100}", (50, 70),cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
    
        devices = AudioUtilities.GetSpeakers()
        interface = devices.Activate(
            IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        volume = cast(interface, POINTER(IAudioEndpointVolume))
 
        currentVolumeDb = volume.GetMasterVolumeLevel()
        vol_range = volume.GetVolumeRange()
        vol = np.interp(vol, [0, 1], [vol_range[0], vol_range[1]]) 
        volume.SetMasterVolumeLevel(vol, None)
    except:
        pass
    cv2.imshow("Image", img)
    cv2.waitKey(1)
print(max_dist)

In [ ]:
!pip install pycaw

In [6]:

#

0.0